In [ ]:
import model_based_mvpa as mbmvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.utils.func import *
from model_based_mvpa.models.model import *
from pathlib import Path
import numpy as np

In [ ]:
root = Path('/data2/project_model_based_fmri/ds001882/')
save_path = Path('.')

## PREPROCESS FMRI IMAGE

In [ ]:
X, masked_data, layout = bids_preprocess(root,save_path=save_path,smoothing_fwhm=None, zoom=(2, 2, 2))

## PREPRROCESS EVENT DATA

In [ ]:
def prep_func(row,info):

    new_df = row.copy()
    new_df['subjID'] = info['subject']
    new_df['run'] = info['run']
    if row['delay_left'] >= row['delay_right']:
        new_df['delay_later'] = row['delay_left']
        new_df['delay_sooner'] = row['delay_right']
        new_df['amount_later'] = row['money_left']
        new_df['amount_sooner'] = row['money_right']
        new_df['choice'] = 1 if row['choice'] == 1 else 0
    else:
        new_df['delay_later'] = row['delay_right']
        new_df['delay_sooner'] = row['delay_left']
        new_df['amount_later'] = row['money_right']
        new_df['amount_sooner'] = row['money_left']
        new_df['choice'] = 1 if row['choice'] == 2 else 0
        
    return new_df

In [ ]:
def cond_func(df_row):
    return df_row['agent']==0

In [ ]:
def latent_func(row,info,k,beta):
    
    new_df = row.copy()
    new_df['subjID'] = info['subject']
    new_df['run'] = info['run']
    
    if row['delay_left'] >= row['delay_right']:
        ev_later   = row['money_left'] / (1 + k * row['delay_left'])
        ev_sooner  = row['money_right'] / (1 + k * row['delay_right'])
        utility = ev_later - ev_sooner
    else:
        ev_sooner   = row['money_left'] / (1 + k * row['delay_left'])
        ev_later  = row['money_right'] / (1 + k * row['delay_right'])
        utility = ev_later - ev_sooner
    new_df['modulation'] = utility
    
    return new_df

In [ ]:
par_names = ['k','beta']

In [ ]:
dm_model = 'dd_hyperbolic'

In [ ]:
df_events = None #pd.read_csv(save_path / 'df_events.tsv',sep = '\t',index_col='Unnamed: 0')

In [ ]:
all_ind_pars = None #pd.read_csv(save_path / 'all_ind_pars.tsv',sep = '\t',index_col='Unnamed: 0')

In [ ]:
dm_model, df_events, signals, time_masks = preprocess_events(root, dm_model,   
                                                              latent_func,par_names,
                                                              prep_func,
                                                              cond_func,
                                                              df_events=None,
                                                              all_ind_pars=all_ind_pars,
                                                              use_duration=False,
                                                              layout=None,
                                                              hrf_model='glover',
                                                              save_path=save_path,
                                                              save=True,
                                                              ncore=4,
                                                              time_check=True)


## LOAD DATA

In [ ]:
prep_path = Path('/home/cheoljun/project_model_based_fmri/examples/Delayed-discount_task')
data_path_list = [prep_path / f'X_{i:02d}.npy' for i in range(1,21)]
X = np.concatenate([np.load(data_path) for data_path in data_path_list],0)
X = X.reshape(-1,X.shape[-1])

In [ ]:
y = np.load(prep_path / 'y.npy' , allow_pickle=True)
y = np.concatenate(y,0)
X = X.reshape(-1,X.shape[-1])
y = y.flatten()


In [ ]:
time_mask = np.load(prep_path / 'y_mask.npy', allow_pickle=True)
time_mask = np.concatenate(time_mask,0)
time_mask = time_mask.flatten()


In [ ]:
X = X[time_mask>0]
y = y[time_mask>0]

masked_data = nib.load(prep_path / 'masked_data.nii.gz')

## FIT MVPA MODEL

In [ ]:
from model_based_mvpa.models.regressor import *
from model_based_mvpa.utils.coef2map import *
import time

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))
N = 30
coefs = mlp_regression(X, y,
                       layer_dims=[1024, 1024],
                       activation_func='linear',
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=30,
                       verbose=1)

task_name = f'piva2019_mlp_{N}'
result = get_map(coefs, masked_data, task_name=task_name, map_type='z', save_path='.', smoothing_sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))
N=3
alpha  = 0.0001
coefs = elasticnet(X, y,
               alpha=alpha,
               n_jobs=16,
               N=N,
               verbose=0)

task_name = f'piva2019_elasticnet_{N}_{alpha:.4f}'
result = get_map(coefs, masked_data, task_name=task_name, map_type='z', save_path='.', smoothing_sigma=1)
print(time.strftime('%c', time.localtime(time.time())))